In [ ]:
!pip install camelot-py==0.8.2
!pip install 'PyPDF2<3.0.0'
!apt-get install ghostscript
import camelot
import pandas as pd

In [ ]:
# Read the tables from the PDF
tables = camelot.read_pdf("Sheet2.pdf", pages = "1,2", flavor = 'lattice')

In [ ]:
tables

In [ ]:
def merge_tables_with_matching_headers(tables):
    merged_tables = []
    
    # Iterate through tables
    i = 0
    while i < len(tables):
        current_table = tables[i]  # Get the current table
        
        # Check if there are subsequent tables
        if i + 1 < len(tables):
            next_table = tables[i + 1]
            
            # Compare top two rows of current and next tables beacuse sometimes the table name or empty rows are present in the first row
            top_rows_current = current_table.df.head(2).values.tolist()
            top_rows_next = next_table.df.head(2).values.tolist()
            
            # Check for any matching row
            match_found = False
            matching_row_index = None
            for idx, row_current in enumerate(top_rows_current):
                for row_next in top_rows_next:
                    if row_current == row_next:
                        # Remove the matched header row from the second table
                        next_table.df.drop(idx, inplace=True)
                        matching_row_index = idx
                        match_found = True
                        break
                if match_found:
                    break
            
            if match_found:
                # Merge tables, excluding the matched header row from the second table
                merged_table = pd.concat([current_table.df, next_table.df], ignore_index=True)
                merged_tables.append(merged_table)
                # Skip the next table as it's already merged
                i += 1
            else:
                # No matching header found, keep current table separate
                merged_tables.append(current_table.df)
        else:
            # No subsequent table, keep current table separate
            merged_tables.append(current_table.df)
        
        i += 1
    
    return merged_tables


# Merge tables with matching headers
merged_tables = merge_tables_with_matching_headers(tables)
